In [18]:
from lxml.etree import HTML
from lxml import etree
import requests
import json
import pandas

df = pandas.read_html(
    requests.get("https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists/Finnish/Press_data").text
)[0]

df[df["PoS"] == "noun"]

,word,rank,count,percentage,PoS
6,vuosi,7,344927,0.585992,noun
26,markka,27,113699,0.193162,noun
39,maa,40,96134,0.163321,noun
42,asia,43,90391,0.153564,noun
46,prosentti,47,84397,0.143381,noun
...,...,...,...,...,...
9977,riitti,9978,436,0.000741,noun
9979,ryhtyminen,9980,436,0.000741,noun
9980,sisäpiiri,9981,436,0.000741,noun
9981,tuliaiset,9982,436,0.000741,noun


In [89]:
words = df[df["PoS"] == "noun"]["word"]

In [94]:
import re

doc = {}
for word in words:
    if len(doc) >= 100:
        break
    for table in HTML(requests.get(f"https://en.wiktionary.org/wiki/{word}").text).xpath("//table"):
        if "fi-decl" in table.attrib.get("class", ""):
            doc[word] = {"s": {"nom": word}}
            for span in table.xpath("//span"):
                m = re.match(".*? (\w+)(?:\/\/\w+)?\|(\w)-form-of", span.attrib.get("class", ""))
                if m:
                    case, quant = m.groups()
                    if quant not in doc[word]:
                        doc[word][quant] = {}
                    doc[word][quant][case] = "".join(span.itertext())
            doc[word]["s"]["_stem"] = re.sub('ssa$', '', doc[word]["s"]["ine"])
            doc[word]["p"]["_stem"] = re.sub('ssa$', '', doc[word]["p"]["ine"])

In [95]:
with open("public/nouns.json", "w") as f:
    f.write(json.dumps(doc, indent=4))